## Smoothers

Various Gauss-Seidel-type MPI-parallel, multiplicative smoothers that overlap MPI and communication:
* Regular Gauss-Seidel
* Block-Gauss-Seidel
* Dynamic-Block-Gauss-Seidel (quite fast for high-order FEM matrices)

Let us set up a Stokes problem on the unit cube to demonstrate

In [5]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ngsolve import *
from ngsolve.webgui import Draw
import ngsolve as ngs
import netgen as ng
import NgsAMG as amg
from netgen.csg import unit_cube
from usrMtgStuffSeq import StokesHDGDiscretization, MakeFacetBlocks 

def gen_ref_mesh (geo, maxh, nref, comm):
    ngs.ngsglobals.msg_level = 1
    if comm.rank==0:
        ngm = geo.GenerateMesh(maxh=maxh)
        if comm.size > 0:
            ngm.Distribute(comm)
    else:
        ngm = ng.meshing.Mesh.Receive(comm)
    ngm.SetGeometry(geo)
    for l in range(nref):
        ngm.Refine()
    return geo, ngs.comp.Mesh(ngm)

mesh = Mesh(unit_cube.GenerateMesh(maxh=0.2))
for k in range(1):
    mesh.ngmesh.Refine()
(V, a, f, u) = StokesHDGDiscretization(mesh, order=4, wall="", inlet="", outlet=".*", nu=1e0, div_div_pen=0)
a.Assemble()

# blocks to use for smoothers later
blocks = MakeFacetBlocks(V, V.FreeDofs(True))

(globally) created 12016 facet-blocks of average size 32.92276964047936


#### How to use NgsAMG smoothers

Drop-in replacementfor NGSolve **multiplicative** GS/BGS smoothers, additive not supported, some additional optional parameters in Smooth/SmoothBack.


In [6]:
from ngsolve import *
import NgsAMG as amg

class SmootherAsPrecond (BaseMatrix):
    def __init__(self, smoother, mat, ngsSmoother=True):
        super(SmootherAsPrecond, self).__init__()
        self.ngsSmoother = ngsSmoother # smooth with residuum
        self.A = mat
        self.S = smoother
        self.res = self.S.CreateColVector()
    def IsComplex(self):
        return False
    def Height(self):
        return self.S.height
    def Width(self):
        return self.S.width
    def CreateColVector(self):
        return self.S.CreateColVector()
    def CreateRowVector(self):
        return self.S.CreateRowVector()
    def MultAdd(self, scal, b, x):
        self.Mult(b, self.xtemp)
        x.data += scal * self.xtemp
    def MultTransAdd(self, scal, b, x):
        self.MultAdd(scal, b, x)
    def MultTrans(self, b, x):
        self.Mult(b, x)
    def Mult(self, b, x):
        x[:] = 0.0
        if not self.ngsSmoother:
            # update residual with forward smooth
            self.res.data = b
            self.S.Smooth(x, b, self.res, x_zero=True, res_updated=True, update_res=True)
            self.S.SmoothBack(x, b, self.res, x_zero=False, res_updated=True, update_res=False)
        else:
            self.S.Smooth(x, b)
            self.S.SmoothBack(x, b)

#### Gauss-Seidel & Block-Gauss-Seidel

Limitation for Block-smoothers in parallel:
 * Blocks may not cross MPI subdomain boundaries
 * Each DOF is owned by the master rank, it will only be included in blocks on that rank

Some examples:
 * Works: blocks of all cell/face/edge/vertex-DOFs
 * Does not work: face/edge-patch, facet-plus-cells
 * Does not work as expected: element, i.e. cell-plus-face-plus-edge-plus-vertex; Master of each DOF will own it, no update from other ranks!



In [9]:
from usrMtgStuffSeq import TestSmoother

# NGSolve built-in smoothers
gs = a.mat.CreateSmoother(V.FreeDofs(True))
bgs = a.mat.CreateBlockSmoother(blocks)

# NgsAMG hybrid smoothers - MPI-parallel & communication overlapping
hybGS = amg.CreateHybridGSS(mat=a.mat,freedofs=V.FreeDofs(True))
hybBGS = amg.CreateHybridBlockGSS(mat=a.mat,blocks=blocks)

TestSmoother(gs, a.mat, True, "NgSolve-GS")
TestSmoother(hybGS, a.mat, True, "NgsAMG-GS")

TestSmoother(bgs, a.mat, True,    "NgSolve-Block-GS")
TestSmoother(hybBGS, a.mat, True, "NgsAMG-Block-GS")


Testing smoother NgSolve-GS:
  If used as preconditioner:
      lam min:   0.0021900969266990524
      lam max:   0.9983676557732455
      condition: 455.85546630486374
  sec per smooth forward:   0.09158714450825729
  sec per smooth backward:  0.1404630096913897

Testing smoother NgsAMG-GS:
  If used as preconditioner:
      lam min:   0.002215785280764354
      lam max:   0.9983693235961033
      condition: 450.5713311949194
  sec per smooth forward:   0.09313129201869233
  sec per smooth backward:  0.11365140847969073

Testing smoother NgSolve-Block-GS:
  If used as preconditioner:
      lam min:   0.002228200734036817
      lam max:   0.9997229612537382
      condition: 448.66826672413237
  sec per smooth forward:   0.1013757567672062
  sec per smooth backward:  0.10111431541003672

Testing smoother NgsAMG-Block-GS:
  If used as preconditioner:
      lam min:   0.0023592213595689812
      lam max:   0.9994825264088574
      condition: 423.64932071972174
  sec per smooth forward:  

#### Dynamic Block-Gauss-Seidel

Sparse Matrix and Smoother implementation that exploits repetitive sparsity pattern of HO matrices.


In [10]:
from usrMtgStuffSeq import TestSPMV

A = a.mat
dynA = amg.ConvertDynBlock(a.mat)

TestSPMV(a.mat, "assembled SparseMatrix")
TestSPMV(dynA, "NgsAMG Dyn-Block matrix")



Testing SPMV assembled SparseMatrix:
  sec per spmv:   0.09350949206103294

Testing SPMV NgsAMG Dyn-Block matrix:
  sec per spmv:   0.03545578619640913


In [14]:
from usrMtgStuffSeq import TestSmoother

bgs = a.mat.CreateBlockSmoother(blocks)

dynSM = amg.CreateDynBlockSmoother(a.mat, V.FreeDofs(True)) 

TestSmoother(bgs,    a.mat, True, "NgSolve-Block-GS")
TestSmoother(dynSM, a.mat, True, "NgsAMG DynBlockSmoother")


Testing smoother NgSolve-Block-GS:
  If used as preconditioner:
      lam min:   0.002212788319141263
      lam max:   0.9997247149932802
      condition: 451.7941035504257
  sec per smooth forward:   0.1012224087213601
  sec per smooth backward:  0.10110836922591745

Testing smoother NgsAMG DynBlockSmoother:
  If used as preconditioner:
      lam min:   0.002358245277728683
      lam max:   0.999727202431127
      condition: 423.9284233377127
  sec per smooth forward:   0.041157394404503465
  sec per smooth backward:  0.04360024573639968
